In [ ]:
import os
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser


load_dotenv()
API_KEY = os.getenv("API_KEY")

from langchain_core.prompts import PromptTemplate ,ChatPromptTemplate

In [ ]:
#chain with custom funtions

In [ ]:
#Task 1 Prompt

prompt_template = ChatPromptTemplate.from_messages(
    [
       ("system", "You are a movie reviewer" ),
       ("human", "{input}" )
    ]
)

In [ ]:
#Task 2 LLM

llm_anthropic = ChatAnthropic(
    model="claude-sonnet-4-5",
    api_key=API_KEY,
    max_tokens=1024
)

In [ ]:
from pydantic import BaseModel
from typing import Literal

class llm_schema(BaseModel):
    movie_summary_flg: Literal["Positive", "Negative"]

llm_structured_output = llm_anthropic.with_structured_output(llm_schema)

In [ ]:
#Task 3 STRPaser
str_parser=StrOutputParser()

In [ ]:
#Task 4 [Custom Runnable]
from langchain_core.runnables import RunnableLambda

def dict_maker(text:str)-> dict:
    return {"text":text}

dict_maker_runnable = RunnableLambda(dict_maker)

In [ ]:
#Task -5 [template for Post]

prompt_post_fb = ChatPromptTemplate.from_messages(
    [
       ("system", "You are a news paper repoter for facebook" ),
       ("human", "create a post regarding folowing {text}" )
    ]
)

In [ ]:
prompt_post_nw = ChatPromptTemplate.from_messages(
    [
       ("system", "You are a news paper repoter for news website" ),
       ("human", "create a post regarding folowing {text}" )
    ]
)

In [ ]:
#chain 


In [ ]:
chain = prompt_template | llm_anthropic | str_parser | dict_maker_runnable | prompt_post | llm_anthropic | str_parser

chain.invoke({"input": "what is your war story"})

In [ ]:
def fb_post(text:str)-> str:
   prompt_post_fb = ChatPromptTemplate.from_messages(
    [
       ("system", "You are a news paper repoter for facebook" ),
       ("human", "create a post regarding folowing {text}" )
    ]
    )
    
   llm_anthropic = ChatAnthropic(
    model="claude-sonnet-4-5",
    api_key=API_KEY,
    max_tokens=1024
   )
    
   str_parser=StrOutputParser()
   chain = prompt_post_fb | llm_anthropic | str_parser
   
   return chain

fb_post_maker_runnable = RunnableLambda(fb_post)

In [ ]:
from langchain_core.runnables import RunnableParallel

In [ ]:
chain = ( prompt_template |
        llm_anthropic | 
        str_parser |
        dict_maker_runnable | 
        RunnableParallel( 
            branch= {"fb_post": fb_post_maker_runnable , "news_post": prompt_post_nw | llm_anthropic | str_parser},

        )
)

In [ ]:
chain.invoke({"input": "what is your war story"})

In [ ]:
from IPython.display import display, Markdown

result = chain.invoke({"input": "what is your war story"})

display(Markdown("## 📱 Facebook Post"))
display(Markdown(result['branch']['fb_post']))

display(Markdown("---"))

display(Markdown("## 📰 News Website Post"))
display(Markdown(result['branch']['news_post']))